In [1]:
!pip install -q -U google-generativeai
! pip install -q --upgrade google-generativeai langchain-google-genai chromadb langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.9/273.9 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import google.generativeai as genai
from google.colab import userdata
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from IPython.display import display
from IPython.display import Markdown
import textwrap


In [3]:

GOOGLE_API_KEY=userdata.get('GEMINI')
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-pro')
response = model.generate_content("tell me how to crack interview")
response.text

'**Preparation**\n\n* **Research the company and role:** Understand the company\'s mission, values, industry, and specific requirements for the position.\n* **Review your resume and cover letter:** Ensure they are accurate, highlight your relevant skills and experience, and tailor them to the job description.\n* **Practice common interview questions:** Prepare answers to standard interview questions such as "Tell me about yourself," "Why are you interested in this role?" and "What are your strengths and weaknesses?"\n* **Consider situational questions:** Anticipate questions that require you to describe how you handled specific work situations and be prepared to provide concrete examples.\n* **Prepare industry-specific questions:** Research the industry and prepare questions to ask the interviewer that demonstrate your knowledge and interest.\n\n**During the Interview**\n\n* **Be punctual and professional:** Arrive on time, dress appropriately, and maintain a positive demeanor.\n* **Ma

# Data Loading

In [6]:
df = pd.read_csv('March-26-2024-MEDIQA-CORR-Official-Test-Set.csv',encoding='latin-1')
# train_data=pd.read_json('merged_data.jsonl',lines=True)

Get Embeddings



# Pipeline

In [7]:
def gemini_inference_pipeline(text):
  '''this function acts as an NER model '''
    prompt = f'You are a NER model for detecting pathogens/disease names in last sentence of given text. .: {text}'
    response = model.generate_content(prompt)
    # print("Response--->", response.text)
    if response  :
        text_content = response.parts[0].text
        if text_content and text_content[0] == 'I':
            return "----"
        return text_content if text_content else "----"
    else:
        return "----"

In [16]:
def get_mask_text(input_text):
  '''This function masks any disease/pathogen/disorder'''
  word_to_be_masked=gemini_inference_pipeline(input_text)
  word_to_be_masked.replace("-", "")
  new_text=input_text.replace(word_to_be_masked,"[MASK]")
  return new_text,word_to_be_masked


def find_sentence_with_word(allSentences, wrong_word):
    '''this function helps to map the sentence id of masked_word(i.e wrong word)'''
    sentences = allSentences.split('\n')
    for i, sentence in enumerate(sentences):
      # print(i,":",sentence)
      if wrong_word in sentence:
        return i # Return the index
    # print("Not found")
    return -1

def get_disease(masked_text):
    ##PaLMyera Model Code
    #Refer PalmYra code
    #this part of code is executed on kaggle notebook
  disease="Tuberculosis"     #the model returns one word correct currently set to dummy
  return disease

In [17]:
#TEST
masked_text=get_mask_text("A 29-year-old internal medicine resident presents to the emergency department with complaints of fevers, diarrhea, abdominal pain, and skin rash for 2 days. He feels fatigued and has lost his appetite. On further questioning, he says that he returned from his missionary trip to Brazil last week. He is excited as he talks about his trip. Besides a worthy clinical experience, he also enjoyed local outdoor activities, like swimming and rafting. His past medical history is insignificant. The blood pressure is 120/70 mm Hg, the pulse is 100/min, and the temperature is 38.3 C (100.9 F). On examination, there is a rash on the legs. Patient's symptoms are suspected to be due to hepatitis A. The rest of the examination is normal.")

In [18]:
#MASKED TEST OUPUT
masked_text


("A 29-year-old internal medicine resident presents to the emergency department with complaints of fevers, diarrhea, abdominal pain, and skin rash for 2 days. He feels fatigued and has lost his appetite. On further questioning, he says that he returned from his missionary trip to Brazil last week. He is excited as he talks about his trip. Besides a worthy clinical experience, he also enjoyed local outdoor activities, like swimming and rafting. His past medical history is insignificant. The blood pressure is 120/70 mm Hg, the pulse is 100/min, and the temperature is 38.3 C (100.9 F). On examination, there is a rash on the legs. Patient's symptoms are suspected to be due to [MASK]. The rest of the examination is normal.",
 'hepatitis A')

In [10]:
def get_output(index,df=df):
  '''Given the index of dataframe, this function gets the corrected sentence along with its id and error flag'''
  ip_text=df['Text'][index]
  ip_all_sentences=df['Sentences'][index]
  masked_text,wrong_word         = get_mask_text(ip_text)
  corr_word                      = get_disease(masked_text)
  corr_text                      = ip_text.replace(wrong_word,corr_word)
  corr_text_in_sentence_id_format= ip_all_sentences.replace(wrong_word,corr_word)
  sentence_id                    = find_sentence_with_word(df['Sentences'][index], wrong_word)

  corr_sentence                  =corr_text_in_sentence_id_format.split('\n')[sentence_id][2:]

  print("MASKED TEXT:",masked_text)
  print("WRONG WORD :",wrong_word)
  print("CORRECT WORD :",corr_word)
  print("CORRECT SENTENCE :",corr_sentence)

  if(wrong_word.lower() ==corr_word.lower() ):
      return df['Text ID'][index] ,0 , -1, "NA"
  return df['Text ID'][index] , 1, sentence_id, corr_sentence


In [19]:
#TEST OUTPUT
print(get_output(0))

MASKED TEXT: A 29-year-old internal medicine resident presents to the emergency department with complaints of fevers, diarrhea, abdominal pain, and skin rash for 2 days. He feels fatigued and has lost his appetite. On further questioning, he says that he returned from his missionary trip to Brazil last week. He is excited as he talks about his trip. Besides a worthy clinical experience, he also enjoyed local outdoor activities, like swimming and rafting. His past medical history is insignificant. The blood pressure is 120/70 mm Hg, the pulse is 100/min, and the temperature is 38.3 C (100.9 F). On examination, there is a rash on the legs. Patient's symptoms are suspected to be due to [MASK]. The rest of the examination is normal.
WRONG WORD : hepatitis A
CORRECT WORD : Tuberculosis
CORRECT SENTENCE :  Patient's symptoms are suspected to be due to Tuberculosis.
('ms-test-0', 1, 10, " Patient's symptoms are suspected to be due to Tuberculosis.")


Iterating through entire dataframe

In [12]:
correct_sen=[]
total_rows = len(df)
# total_rows = 597

batch_size = 10

In [13]:
import time
def get_final_Output():
  with open("output.txt", "w") as file:
    for i in range(0, total_rows, batch_size):
          end_index = min(i + batch_size, total_rows)
          for j in range(i, end_index):
              time.sleep(0.1)
              try:
                myTuple = get_output(j)
                print(j,"-->",myTuple,"\n")
                correct_sen.append(myTuple)
                line = ' '.join(map(str, inner_tuple))
                file.write(line + "\n")
              except Exception as e:
                inner_tuple=(df['Text ID'][j],0, -1, 'NA')
                line = ' '.join(map(str, inner_tuple))
                file.write(line + "\n")
                correct_sen.append(inner_tuple)


Concluding ...
With this we get the output.txt file wich is our final text file.

--
Please note to refer to kaggle notebook for palmyera



# Masking Code

In [24]:
all_masked_sentences=[]
all_worng_word=[]

In [ ]:
import time
for i in range(0,len(df)):
  time.sleep(0.25) #sleeper added to ensure smooth API call
  try:
    masked_text,wrong_word=get_mask_text(df["Sentences"][i])
    wrong_word=wrong_word.replace("-","")
    print("---->",wrong_word,"-%%%---",masked_text)
    all_masked_sentences.append(masked_text)
    all_worng_word.append(wrong_word)
  except Exception as e:
    print("Some error",e)
    all_masked_sentences.append(df["Sentences"][i])
    all_worng_word.append("NONE")

----> hepatitis A -%%%--- 0 A 29-year-old internal medicine resident presents to the emergency department with complaints of fevers, diarrhea, abdominal pain, and skin rash for 2 days.
1 He feels fatigued and has lost his appetite.
2 On further questioning, he says that he returned from his missionary trip to Brazil last week.
3 He is excited as he talks about his trip.
4 Besides a worthy clinical experience, he also enjoyed local outdoor activities, like swimming and rafting.
5 His past medical history is insignificant.
6 The blood pressure is
7 120/70 mm
8 Hg, the pulse is 100/min, and the temperature is 38.3 C (100.9 F).
9 On examination, there is a rash on the legs.
10 Patient's symptoms are suspected to be due to [MASK].
11 The rest of the examination is normal.

----> Schistosoma mansoni -%%%--- 0 A 29-year-old internal medicine resident presents to the emergency department with complaints of fevers, diarrhea, abdominal pain, and skin rash for 2 days.
1 He feels fatigued and has 

ERROR:tornado.access:500 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 12590.34ms


Some error 500 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: TypeError: Failed to fetch
----> dysarthiaclumsy hand syndrome -%%%--- 0 A 57-year-old woman is brought to the emergency department by her husband with complaints of sudden-onset slurring for the past hour.
1 She is also having difficulty holding things with her right hand.
2 She denies fever, head trauma, diplopia, vertigo, walking difficulties, nausea, and vomiting.
3 Past medical history is significant for type 2 diabetes mellitus, hypertension, and hypercholesterolemia for which she takes a baby aspirin, metformin, ramipril, and simvastatin.
4 She has a 23-pack-year cigarette smoking history.
5 Her blood pressure is
6 148/96 mm
7 Hg, the heart rate is 84/min, and the temperature is 37.1 C (98.8 F).
8 On physical examination, extraocular movements are intact.
9 The patient is dysarthric, but her higher mental functions are intact.
10 There is a ri

ERROR:tornado.access:500 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 45503.63ms


Some error 500 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: TypeError: Failed to fetch
---->  hypertrophic cardiomyopathy -%%%--- 0 A 23-year-old man presents with sudden loss of consciousness while pitching in a baseball game.
1 There is no history of injury.
2 Consciousness is regained after cardiopulmonary resuscitation.
3 Past medical history is negative for any neurological and cardiovascular problems.
4 Physical examination reveals a prominent A wave on the jugular venous pulse and a double apical impulse.
5 There are no audible murmurs.
6 An S4 is present.
7 Patient is diagnosed with hypertrophic cardiomyopathy.

----> There are no pathogens or diseases mentioned in the text. -%%%--- 0 A 35-year-old woman presents to her family doctor worried that she might have a heart condition.
1 For the past 7 months, she has been having short panic attacks where she feels short of breath, sweaty, and feels like he

In [ ]:
print("MASKED WORD :",all_wrong_word[0])         #these 2 will be added to the dataset
print("MASKED SENTENCE :",all_masked_sentences[0]) #the new dtaset formed is used as masked_dataset

#OUTPUT


# MASKED WORD : hepatitis A
# MASKED SENTENCE :A 29-year-old internal medicine resident presents to the emergency department with complaints of fevers, diarrhea, abdominal pain, and skin rash for 2 days.
# 1 He feels fatigued and has lost his appetite.
# 2 On further questioning, he says that he returned from his missionary trip to Brazil last week.
# 3 He is excited as he talks about his trip.
# 4 Besides a worthy clinical experience, he also enjoyed local outdoor activities, like swimming and rafting.
# 5 His past medical history is insignificant.
# 6 The blood pressure is
# 7 120/70 mm
# 8 Hg, the pulse is 100/min, and the temperature is 38.3 C (100.9 F).
# 9 On examination, there is a rash on the legs.
# 10 Patient's symptoms are suspected to be due to [MASK].
# 11 The rest of the examination is normal.


In [ ]:
df['new_masked_text']=all_masked_sentences
df['masked_word']=all_worng_word

In [ ]:
df.head(2)

In [ ]:
df.to_csv('masked_dataset.csv', index=False)
